In [ ]:
import sys
import os

os.chdir("..")
from modules.data.download import download_data
download_data("2025-07-01", "2025-07-01")

In [1]:
import sys
import os

os.chdir("..")

from modules.points.generate import generate_points
generate_points(batch_size=1)